# Goal
### Overall Goal
See how useful Spark is to explore a large data set.
__*PUDF_base_all_tab.txt*__ is 10 GB and 18 M observations on over 250 features.

### Specific to this notebook
Explore how clean nine (9) columns of interest are:
1. PROVIDER_NAME
2. ADMIT_WEEKDAY
3. pat_age
4. RACE
5. ETHNICITY
6. FIRST_PAYMENT_SRC
7. SECONDARY_PAYMENT_SRC
8. ADMITTING_DIAGNOSIS
9. TYPE_OF_ADMISSION

In [1]:
from pyspark.sql.types import StructType
from os import path, getcwd
import sys

In [2]:
def get_distinct_number(df, col):
    if df is None or col is None:
        print("The dataframe or col is invalid")
        return sqlContext.createDataFrame(sc.emptyRDD(), StructType([]))
    elif col not in df.columns:
        print("The column '{}' is not in the dataframe." \
             .format(col))
        return sqlContext.createDataFrame(sc.emptyRDD(), df.schema)
    else:
        return df.groupBy(col).count().count()

def get_topN_group(df, col, sort=True, highest_first=True):
    if df is None or col is None:
        print("The dataframe or col is invalid")
        return 
    elif col not in df.columns:
        print("The column '{}' is not in the dataframe." \
             .format(col))
    else:
        if sort:
            return df.groupBy(col).count().orderBy("count", ascending = not highest_first)
        else:
            return df.groupBy(col).count()   

In [ ]:
texas_df = spark.read.format('com.databricks.spark.csv') \
    .options(header='true', inferschema='true', delimiter='\t')\
    .load(path.join(getcwd(), "..", "data", "PUDF_base_all_tab.txt"))

# Munging

# Data Exploration
Looking at completeness of variables in __*goal*__

### All columns

In [26]:
texas_df_columns = texas_df.columns
print(texas_df_columns)

['discharge_qtr', 'thcic_id', 'PROVIDER_NAME', 'FAC_TEACHING_IND', 'FAC_PSYCH_IND', 'FAC_REHAB_IND', 'FAC_ACUTE_CARE_IND', 'FAC_SNF_IND', 'FAC_LONG_TERM_AC_IND', 'FAC_OTHER_LTC_IND', 'FAC_PEDS_IND', 'SPEC_UNIT_1', 'SPEC_UNIT_2', 'SPEC_UNIT_3', 'SPEC_UNIT_4', 'SPEC_UNIT_5', 'ENCOUNTER_INDICATOR', 'SEX_CODE', 'TYPE_OF_ADMISSION', 'SOURCE_OF_ADMISSION', 'PAT_STATE', 'PAT_ZIP', 'PAT_COUNTRY', 'county', 'public_health_region', 'ADMIT_WEEKDAY', 'LENGTH_OF_STAY', 'pat_age', 'PAT_STATUS', 'RACE', 'ETHNICITY', 'FIRST_PAYMENT_SRC', 'SECONDARY_PAYMENT_SRC', 'TYPE_OF_BILL', 'private_amount', 'semi_private_amount', 'ward_amount', 'icu_amount', 'ccu_amount', 'other_amount', 'pharm_amount', 'medsurg_amount', 'dme_amount', 'used_dme_amount', 'pt_amount', 'ot_amount', 'speech_amount', 'it_amount', 'blood_amount', 'blood_adm_amount', 'or_amount', 'lith_amount', 'card_amount', 'anes_amount', 'lab_amount', 'rad_amount', 'mri_amount', 'op_amount', 'er_amount', 'ambulance_amount', 'pro_fee_amount', 'organ_a

### LOS & PAT_AGE

In [5]:
print("There are non-numerics for LOS and POS.\n" + \
      "# LOS with value 'LENGTH_OF_STAY': {}.\n "\
      .format(
        texas_df.filter(texas_df["LENGTH_OF_STAY"] == 'LENGTH_OF_STAY').count()) + \
      "# LOS with value '*': {}. \n" \
      .format(
      texas_df.filter(texas_df["LENGTH_OF_STAY"] == '*').count()) + \
      "# PAT_AGE with value '*': {}.\n" \
      .format(
      texas_df.filter(texas_df["PAT_AGE"] == '*').count()) + \
      "# PAT_AGE with value 'ZZ': {}." \
      .format(texas_df.filter(texas_df["PAT_AGE"] == 'ZZ').count()))

There are non-numerics for LOS and POS.
# LOS with value 'LENGTH_OF_STAY': 3.
 # LOS with value '*': 8181. 
# PAT_AGE with value '*': 983.
# PAT_AGE with value 'ZZ': 152283.


In [36]:
for col in ["LENGTH_OF_STAY", "pat_age"]:
    print("# distinct '{}': {}.\n".format(col, get_distinct_number(texas_df, col)))

# distinct 'LENGTH_OF_STAY': 604.

# distinct 'pat_age': 48.



In [5]:
%timeit -n1 -r1 texas_df.describe("LENGTH_OF_STAY", "PAT_AGE").show()

+-------+-----------------+------------------+
|summary|   LENGTH_OF_STAY|           PAT_AGE|
+-------+-----------------+------------------+
|  count|         18114764|          18147843|
|   mean|2.280949958520556|11.599688703625358|
| stddev|9.438761332878363| 4.850573319201647|
|    min|                *|                 *|
|    max|   LENGTH_OF_STAY|                ZZ|
+-------+-----------------+------------------+

1 loop, best of 1: 1min 20s per loop


### Patient Gender

In [25]:
%timeit -n1 -r1 texas_df.groupBy("SEX_CODE").count().orderBy("count", ascending=False).show(20)

+--------+-------+
|SEX_CODE|  count|
+--------+-------+
|       F|1661639|
|       M|1027482|
|    null| 992226|
|   88888| 102457|
|   78521|  70316|
|   78572|  63415|
|       *|  60799|
|   79936|  51113|
|   78501|  50928|
|   78577|  49470|
|   75217|  48805|
|   78539|  48298|
|   75228|  42459|
|   78596|  41771|
|   78550|  41233|
|   75211|  40880|
|   78520|  40843|
|   75216|  40407|
|   77084|  39612|
|   78207|  39258|
+--------+-------+
only showing top 20 rows

1 loop, best of 1: 7min 37s per loop


In [22]:
%timeit -n1 -r1 texas_df.groupBy("LENGTH_OF_STAY").count().orderBy("count", ascending=False).show(20)

+--------------+--------+
|LENGTH_OF_STAY|   count|
+--------------+--------+
|             2|10877316|
|             1| 4410786|
|          0002|  735987|
|          0003|  467292|
|          0001|  466910|
|          0004|  268435|
|          0005|  174801|
|          0006|  130284|
|          0007|  106240|
|          0008|   76916|
|          0009|   56394|
|          0010|   44955|
|          null|   43369|
|          0011|   36529|
|          0012|   28968|
|          0013|   25607|
|          0014|   24318|
|          0015|   19085|
|          0016|   15173|
|          0017|   13030|
+--------------+--------+
only showing top 20 rows

1 loop, best of 1: 3min 13s per loop


### Admit Diagnosis

In [17]:
## Number of Diagnosis Codes
%timeit -n1 -r1 print(texas_df.groupBy("ADMITTING_DIAGNOSIS").count().count())

11303
1 loop, best of 1: 2min 20s per loop


In [17]:
get_topN_group(texas_df, "ADMITTING_DIAGNOSIS").show(25)

+-------------------+-------+
|ADMITTING_DIAGNOSIS|  count|
+-------------------+-------+
|               null|5032634|
|               4019| 715086|
|               V270| 308122|
|              25000| 296101|
|              V3000| 240512|
|               4280| 239354|
|               2724| 215109|
|              41401| 188946|
|               3051| 165513|
|               5990| 155951|
|              53081| 143961|
|              42731| 140364|
|               2859| 135045|
|               V221| 125632|
|               2449| 123831|
|                496| 115170|
|              V3001| 110404|
|               2761| 100086|
|                486|  98278|
|               2768|  94414|
|               V053|  91897|
|              78650|  91359|
|              40391|  81074|
|               2720|  76036|
|              27651|  72060|
+-------------------+-------+
only showing top 25 rows



### Patient Race

In [18]:
print(get_distinct_number(texas_df, "ETHNICITY"))

52489


In [19]:
%timeit -n1 -r1 texas_df.groupBy("ETHNICITY").count().orderBy("count", ascending=False).show(20)

+---------+-------+
|ETHNICITY|  count|
+---------+-------+
|     0.00|8905012|
|        2|2043158|
|        1| 765130|
|  1182.00|  32114|
|  2400.00|  28429|
|  1773.00|  27205|
|  2000.00|  26900|
|   591.00|  23950|
|  1604.00|  21883|
|  1528.00|  21617|
|  2160.00|  19552|
|  1850.00|  19396|
|  1406.00|  19042|
|  1108.00|  18642|
|  1456.00|  18492|
|  2364.00|  17633|
|  1000.00|  16816|
|  1624.00|  16401|
|  3000.00|  16201|
|  3240.00|  15961|
+---------+-------+
only showing top 20 rows

1 loop, best of 1: 2min 15s per loop


In [21]:
%timeit -n1 -r1 texas_df.groupBy("RACE").count().orderBy("count", ascending=False).show(20)

+----+--------+
|RACE|   count|
+----+--------+
| 111|15198686|
|   4| 1802112|
|   5|  600784|
|   3|  354070|
|   2|   45705|
| 121|   40574|
| 211|   37878|
| 110|   27573|
| 131|   16543|
| 181|   11919|
|   1|   10989|
| 114|    5890|
|null|    4777|
| 641|     149|
| 711|      96|
| 210|      88|
| 281|      77|
| 171|      46|
| 134|      37|
|   *|      23|
+----+--------+
only showing top 20 rows

1 loop, best of 1: 59.1 s per loop


### Providers

In [20]:
## Number of Diagnosis Codes
%timeit -n1 -r1 print(texas_df.groupBy("PROVIDER_NAME").count().count())

778
1 loop, best of 1: 1min 33s per loop


In [16]:
get_topN_group(texas_df, "PROVIDER_NAME").show(20)

+--------------------+------+
|       PROVIDER_NAME| count|
+--------------------+------+
|Parkland Memorial...|350607|
|  Methodist Hospital|308712|
|Baylor University...|266215|
|Memorial Hermann ...|228010|
|St Lukes Episcopa...|212507|
|Harris Methodist-...|194237|
|John Peter Smith ...|185952|
|Ben Taub General ...|185004|
|The Methodist Hos...|183536|
|Scott & White Mem...|179892|
|Presbyterian Hosp...|171138|
|Memorial Hermann ...|169908|
|McAllen Medical C...|167748|
|Medical City Dall...|166803|
|Seton Medical Center|165754|
|Mother Frances Ho...|148905|
| University Hospital|147828|
|Clear Lake Region...|147820|
|Baptist St Anthon...|140277|
|Providence Memori...|137563|
+--------------------+------+
only showing top 20 rows



### Admin Day of the week

In [19]:
get_topN_group(texas_df, "ADMIT_WEEKDAY").show(20)

+-------------+--------+
|ADMIT_WEEKDAY|   count|
+-------------+--------+
|            4|10169480|
|            5| 3624802|
|            3| 2434129|
|            2|  757208|
|            1|  634216|
|            6|  254662|
|            7|  245730|
|         null|   26204|
|            *|   11678|
|         RACE|      21|
|ADMIT_WEEKDAY|       3|
+-------------+--------+



### Type & Source of Admission

In [35]:
for col in ["TYPE_OF_ADMISSION", "SOURCE_OF_ADMISSION"]:
    print("# of unique '{}' is: {}.\n".format(col, get_distinct_number(texas_df, col)))

# of unique 'TYPE_OF_ADMISSION' is: 66.

# of unique 'SOURCE_OF_ADMISSION' is: 269.



In [21]:
for col in ["TYPE_OF_ADMISSION", "SOURCE_OF_ADMISSION"]:
    get_topN_group(texas_df, col).show(20)

+-----------------+--------+
|TYPE_OF_ADMISSION|   count|
+-----------------+--------+
|             null|12444656|
|               US| 2852878|
|                1| 1046899|
|                3|  843946|
|                2|  551669|
|                4|  357630|
|               MX|   26722|
|                9|   11703|
|                5|    6531|
|               CL|    6256|
|                *|    2917|
|               KW|     827|
|               CA|     807|
|               SA|     578|
|               AE|     388|
|               GT|     306|
|               HN|     282|
|               VE|     263|
|               PE|     236|
|               PR|     202|
+-----------------+--------+
only showing top 20 rows

+-------------------+-------+
|SOURCE_OF_ADMISSION|  count|
+-------------------+-------+
|                201|2329758|
|                113|1453804|
|                  1|1115478|
|                  7|1057671|
|                439| 998204|
|                029| 978970|
|       

### Payment Source

In [31]:
payment_columns = ["FIRST_PAYMENT_SRC", "SECONDARY_PAYMENT_SRC"]
[col in texas_df_columns for col in payment_columns]

[True, True]

In [34]:
for col in payment_columns:
    print("# of unique '{}' is: {}.\n".format(col, get_distinct_number(texas_df, col)))

# of unique 'FIRST_PAYMENT_SRC' is: 128632.

# of unique 'SECONDARY_PAYMENT_SRC' is: 436.



In [33]:
for col in payment_columns:
    get_topN_group(texas_df, col).show(20)

+-----------------+-------+
|FIRST_PAYMENT_SRC|  count|
+-----------------+-------+
|             0.00|8407482|
|               MA| 876556|
|               MC| 607329|
|               12| 333160|
|               **| 284319|
|               CI| 211086|
|               HM| 189068|
|               BL| 184924|
|           570.00|  44882|
|           940.00|  44560|
|               16|  42895|
|          1410.00|  40853|
|          2400.00|  37779|
|           470.00|  34157|
|          1200.00|  32743|
|               11|  28575|
|          1400.00|  27982|
|           600.00|  25565|
|          1050.00|  25485|
|               CH|  24857|
+-----------------+-------+
only showing top 20 rows

+---------------------+--------+
|SECONDARY_PAYMENT_SRC|   count|
+---------------------+--------+
|                 0.00|15321575|
|                 null| 1886829|
|                   CI|  269486|
|                   MC|  217141|
|                   **|  154312|
|                   BL|   78025|
|    

# Scrap

In [12]:
%timeit -r1 -n1 print(get_topN_group(texas_df, "RACE", False).count())

41
1 loop, best of 1: 1min 4s per loop


In [11]:
%timeit -r1 -n1 print(texas_df.select("RACE").distinct().count())

41
1 loop, best of 1: 1min 3s per loop
